In [85]:
# SELECT PLAY HERE:
play = 4083
game = 2022100906

In [86]:
import pandas as pd
import numpy as np

In [87]:
plays = pd.read_csv('../data/tracking_week_5.csv')

# Grab only items that are before and during the snap
plays = plays[plays['frameType'].isin(['BEFORE_SNAP', 'BALL_SNAP'])]

# drop rows that are about the football
plays = plays[plays['displayName'] != 'football']

In [88]:
# Find the player with the maximum 's' within each 'gameId' and 'playId'
max_s_players = plays.loc[plays.groupby(['gameId', 'playId'])['s'].idxmax()]
# Get the row before the maximum 's' within each 'gameId' and 'playId'
previous_rows = plays.loc[plays.index.isin(max_s_players.index - 1)]
# Combine the rows
max_s_players = pd.concat([previous_rows, max_s_players]).sort_index()

In [89]:
max_s_players['orientation_sin'] = np.sin(np.radians(max_s_players['o']))
max_s_players['orientation_cos'] = np.cos(np.radians(max_s_players['o']))
max_s_players['forward_velocity'] = max_s_players['s'] * max_s_players['orientation_cos']
max_s_players['sideways_velocity'] = max_s_players['s'] * max_s_players['orientation_sin']

In [90]:
def determine_player_direction(df):
    # Ensure the dataframe is sorted by gameId, playId, and frameId
    df = df.sort_values(by=['gameId', 'playId', 'frameId'])
    
    # Calculate the difference in y values between consecutive frames
    df['y_diff'] = df.groupby(['gameId', 'playId'])['y'].diff()
    
    conditions = [
        (df['o'] > 180) & (df['y_diff'] > 0),
        (df['o'] > 180) & (df['y_diff'] < 0),
        (df['o'] < 180) & (df['y_diff'] > 0),
        (df['o'] < 180) & (df['y_diff'] < 0)
    ]
    
    # Define the corresponding values for the new column
    choices = ['right', 'left', 'left', 'right']
    
    # Create the new column based on the conditions
    df['player_direction'] = np.select(conditions, choices, default=np.nan)
    
    return df

# Apply the function to the max_s_players dataframe
max_s_players = determine_player_direction(max_s_players)

In [91]:
max_s_players['player_going_right'] = pd.get_dummies(max_s_players['player_direction'])['right']
max_s_players.drop(columns=['player_direction'], inplace=True)

In [92]:
# Select play here
max_s_players = max_s_players[max_s_players['gameId'] == game]
max_s_players = max_s_players[max_s_players['playId'] == play]

In [93]:
df = max_s_players[['playId', 'gameId', 'frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'orientation_sin', 'orientation_cos', 'forward_velocity', 'sideways_velocity', 'y_diff', 'player_going_right']]

In [ ]:
df = df.drop(columns=['playId', 'gameId'])
df.dropna(inplace=True)

In [97]:
df.head()

,frameId,x,y,s,a,dis,o,dir,orientation_sin,orientation_cos,forward_velocity,sideways_velocity,y_diff,player_going_right
3944161,73,46.5,25.48,3.98,0.05,0.4,21.42,23.31,0.365202,0.930928,3.705095,1.453503,0.37,False


In [95]:
import joblib

# Load the model
rf_classifier = joblib.load('../models/rf_classifier.pkl')

In [ ]:
X = df[['frameId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'orientation_sin', 'orientation_cos', 'forward_velocity', 'sideways_velocity', 'y_diff', 'player_going_right']]

# Predict using the model
predictions = rf_classifier.predict(X)

# Print the result
print(predictions)

[0.]
